# RAPIDS와 GPU를 사용한 데이터 전처리

이 노트북에서는 고차원 항공사 데이터의 일부인 미국 교통통계국(U.S. Bureau of Transportation Statistics)에서 배포하는 항공 서비스 품질 성과 데이터셋(Airline Service Quality Performance dataset, 1987-2021년, https://www.bts.dot.gov/browse-statistical-products-and-data/bts-publications/airline-service-quality-performance-234-time)을 사용합니다.

이 데이터셋은 오픈 소스이며 미국 교통통계국에서 지속적으로 제공하고 있습니다.

교통통계국은 매달 전월의 모든 항공편 정보를 담은 새로운 CSV 파일을 게시합니다. 강력한 머신러닝 모델을 학습시키려면 여러 해에 걸친 데이터를 결합하여 학습 데이터셋으로 사용하는 것이 좋습니다. 이 실습에서는 설명 목적으로 10일치의 데이터만 사용합니다. 하지만 이 스크립트는 cuDF를 사용하여 데이터를 로드하고 전처리하므로 대량의 데이터를 다루더라도 빠르게 실행될 것입니다.

항공편 데이터 외에도 각 공항의 메타데이터와 지리적 좌표가 포함된 파일과 각 항공사의 코드 매핑이 포함된 파일을 다운로드하게 됩니다. 항공사와 공항은 거의 변경되지 않으므로 이러한 파일은 고정되어 있으며 매달 업데이트되지 않습니다. 하지만 이 파일에는 나중에 전체 항공사 데이터 세트에 매핑하는 데 필요한 정보가 포함되어 있습니다. (데이비드 메긴슨, "airports.csv", OurAirports 배포, 2021년 8월 2일, https://ourairports.com/data/airports.csv)

## 환경 변수 가져오기

작업을 제출하기 전에 작업 공간 및 환경과 같은 필요한 모든 환경 변수를 확보해야 합니다.

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()

In [ ]:
# RAPIDS/cuDF 환경 준비
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

env_name = "rapids-cudf-23_08"
existing_envs = Environment.list(workspace=ws)

if env_name in existing_envs:
    rapidsai_env = Environment.get(workspace=ws, name=env_name)
    print(f"✓ 기존 환경 사용: {rapidsai_env.name}")
else:
    deps = CondaDependencies()
    deps.add_channel("rapidsai")
    deps.add_channel("nvidia")
    deps.add_channel("conda-forge")
    deps.add_conda_package("python=3.10")
    deps.add_conda_package("cudf=23.08")
    deps.add_conda_package("cudatoolkit=11.8")
    deps.add_pip_package("mlflow")

    rapidsai_env = Environment(name=env_name)
    rapidsai_env.python.conda_dependencies = deps
    rapidsai_env.register(workspace=ws)

    print(f"✓ 새 환경 생성/등록 완료: {rapidsai_env.name}")
    print("처음 1회는 이미지 빌드에 시간이 걸릴 수 있습니다.")

## 구성 정의 및 실행 제출

필요한 변수를 모두 정의했으므로 이제 스크립트 실행 구성을 정의하고 실행을 제출할 수 있습니다.

**경고!** 아래 코드를 실행하기 전에 compute_target 변수 값을 컴퓨팅 클러스터 이름으로 변경하십시오!
00-setup.md 을 따라 수행하셨다면 `cc-aml-gpu` 를 입력하십시오.

In [ ]:
# 사용 가능한 Compute Cluster 확인
from azureml.core.compute import ComputeTarget

print("🔍 현재 Workspace의 모든 Compute 리소스:")
print("-" * 60)

for compute_name in ComputeTarget.list(ws):
    compute = ComputeTarget(workspace=ws, name=compute_name)
    print(f"  이름: {compute_name}")
    print(f"    타입: {compute.type}")
    print(f"    상태: {compute.status}")
    print()

print("💡 위의 목록에서 'AmlCompute' 타입 중 하나를 아래 compute_target에 입력하세요.")

In [ ]:
from azureml.core import ScriptRunConfig

src = ScriptRunConfig(source_directory='script',
                      script='preprocess-rapids.py',
                      compute_target="<compute_target>",
                      environment=rapidsai_env)

전처리 과정에서 어떤 작업이 수행되는지 알아보려면 `script` 폴더에 있는 `preprocess-rapids.py` 스크립트를 살펴보세요.

다음 셀은 스크립트 실행을 시작합니다. 먼저 컴퓨팅 클러스터의 노드 수가 0개에서 확장되어야 합니다. 노드가 사용 가능해지면 스크립트가 실행됩니다. 스크립트 실행은 빠르며, 실행 시간은 실행 후 **Assets** → **Jobs** → **Experiment** 보고서의 **Details** 탭에서 확인할 수 있습니다.

In [ ]:
from azureml.core import Experiment

run = Experiment(ws,'preprocess-data').submit(src)
run.wait_for_completion(show_output=True)

스튜디오에서 새 실행이 시작되어 진행 중이라는 알림을 받게 됩니다.

또는 **Experiments** 탭으로 이동하여 `preprocess-data` 실험을 찾을 수도 있습니다.

실행이 완료되면 **Metrics** 탭에서 처리된 데이터 양을 확인할 수 있습니다. **Details** 탭에서는 실행에 소요된 시간을 볼 수 있습니다. 처리된 데이터는 **outputs** 폴더의 **Outputs+logs**에서 확인할 수 있습니다.